# Iterativne metode

---

Za velike sustave, a posebno za sustave s malom ispunom (malo elemenata različitih od nule), te ukoliko je matrica sustava __strogo dijagonalno dominantna__, rješenje se može brzo naći __iterativnim metodama__
(vidi [Numerička matematika, poglavlje 3.8][RS04]):

__Definicija.__ Funkcija $F:\mathbb{R}^n\to \mathbb{R}^n$ je __kontrakcija__ ako postoji broj $q<1$ za koji vrijedi

$$
\| F(x)-F(y)\| < q\|x-y\|\qquad \forall x,y.
$$

__Banachov teorem o fiksnoj točki.__
Ako je $F$ kontrakcija, onda niz definiran s

$$ 
x_{k+1}=F(x_k)
$$

konvergira prema jedinstvenom vektoru $\tilde x$ za kojeg vrijedi

$$
\tilde x = F(\tilde x).
$$

$\tilde x$ se zove __fiksna točka__ funkcije $F$. Za pogrešku u $k$-tom koraku vrijede ocjene

$$
\|x_k- \tilde x\| \leq \frac{q}{1-q} \|x_k-x_{k-1}\|
$$

i 

$$
\|x_k- \tilde x\| \leq \frac{q^k}{1-q} \|x_1-x_{0}\|,
$$

pri čemu je druga ocjena bolja. Brzina konvergencije je __linearna__,

$$
\|x_{k+1}-\tilde x\| \leq q\| x_k-\tilde x\|.
$$

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."


## Jacobi-jeva i Gauss-Seidel-ova metoda

Neka je 

$$F(x)=Bx+c,$$

pri čemu je $B$ kvadratna matrica. Tada je

$$
\| F(x)-F(y)\|=\| Bx+c-(By+c)\|=\|B(x-y)\| \leq \|B\| \|x-y\|,
$$

pa je $F$ kontrakcija ako je

$$
 \|B\|=q<1.
$$

Neka je zadan sustav  $Ax=b$. Matricu $A$ rastavimo kao

$$
A=D\,(L+I+U)
$$

pri čemu je $D$ dijagonalna matrica, $L$ strogo donje trokutasta matrica i $U$ strogo gornje trokutasta matrica.

### Jacobi-jeva metoda 

Neka je 

$$
B=-(L+U), \quad c=D^{-1}b.
$$


Ako je matrica $A$ _strogo dijagonalno dominantna_, 

$$
\| B\|_{\infty} = \max_i \sum_{{j=1} \atop {j\neq i}}^n \frac{|a_{ij}|}{|a_{ii}|}<1,
$$

onda je preslikavanje $F$ kontrakcija (moguće je uzeti i druge norme) pa niz

$$
x_{k+1}=-(L+U)x_k+c
$$

konvergira prema rješenju sustava $x$.

### Gauss-Seidel-ova metoda 

Neka je 

$$
B=-(I+L)^{-1}U, \quad c=(I+L)^{-1}\, D^{-1}b.
$$

Bez dokaza navodimo sljedeću tvrdnju: ako je matrica $A$ strogo dijagonalno dominantna,
onda je preslikavanje $F$ kontrakcija pa niz

$$
x_{k+1}=-(I+L)^{-1}Ux_k+(I+L)^{-1}D^{-1}b,
$$

odnosno

$$
x_{k+1}=-Lx_{k+1}-Ux_k+D^{-1}b,
$$

konvergira prema rješenju sustava $x$.

In [1]:
function myjacobi{T}(A::Array{T},b::Array{T},x::Array{T})
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    B=-(L+U)
    c=b./D
    q=norm(B,Inf)
    # @show q
    while d>tol
        y=B*x+c
        d=norm(x-y,Inf)
        # @show d
        x=y
    end
    x,d
end

myjacobi (generic function with 1 method)

In [2]:
n=8
A=rand(n,n)
# Napravimo matricu dijagonalno diminantnom
A=A+n*I
b=rand(n)

8-element Array{Float64,1}:
 0.088453 
 0.14392  
 0.0701814
 0.519542 
 0.469315 
 0.358071 
 0.547121 
 0.977004 

In [3]:
# Pocetni vektor
x0=rand(n)

8-element Array{Float64,1}:
 0.18207 
 0.672737
 0.366656
 0.734968
 0.209452
 0.504647
 0.122966
 0.406642

In [4]:
x,d=myjacobi(A,b,x0)

([-0.00768521, 0.00713767, -0.00950371, 0.0530597, 0.0479081, 0.0272049, 0.0532992, 0.106753], 9.611755835692293e-14)

In [5]:
r=A*x-b

8-element Array{Float64,1}:
 2.60653e-13
 1.63036e-13
 2.04836e-13
 2.57572e-13
 1.8946e-13 
 2.84273e-13
 2.6712e-13 
 3.35842e-13

In [6]:
# Provjerimo i normu relativnog reziduala
norm(r)/(vecnorm(A)*norm(x))

2.0467094622975933e-13

In [7]:
function mygaussseidel{T}(A::Array{T},b::Array{T},x::Array{T})
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    # B=-inv(I+L)*U
    B=-(I+L)\U
    c=(I+L)\(b./D)
    # @show norm(U,Inf)
    y=Array{Float64}(n)
    while d>tol
        y=B*x+c
        d=norm(x-y)
        x=y
    end
    x,d
end

mygaussseidel (generic function with 1 method)

In [8]:
x,d=mygaussseidel(A,b,x0)

([-0.00768521, 0.00713767, -0.00950371, 0.0530597, 0.0479081, 0.0272049, 0.0532992, 0.106753], 1.2593380629283113e-13)

In [9]:
A*x-b

8-element Array{Float64,1}:
 -4.27991e-14
 -2.62013e-14
 -4.03982e-14
 -5.16254e-14
 -1.7486e-14 
 -1.18794e-14
 -4.77396e-15
  2.22045e-16

Izmjerimo brzinu za veće matrice:

In [10]:
n=1024
A=rand(n,n)
# Napravimo matricu dijagonalno dominantnom
A=A+n*I
b=rand(n)
# Pocetni vektor
x0=rand(n)

1024-element Array{Float64,1}:
 0.07212  
 0.735047 
 0.386647 
 0.652678 
 0.064069 
 0.226201 
 0.278475 
 0.303908 
 0.878075 
 0.825688 
 0.79909  
 0.866191 
 0.921129 
 ⋮        
 0.0524838
 0.90724  
 0.251845 
 0.484417 
 0.347099 
 0.942431 
 0.145327 
 0.154361 
 0.0100958
 0.909879 
 0.517155 
 0.734318 

In [12]:
@time mygaussseidel(A,b,x0);

  0.182771 seconds (77 allocations: 64.413 MiB, 48.96% gc time)
